In [38]:
import os
import string
from datetime import datetime
import numpy as np
import itertools

In [39]:
def timediff(t1,t2):
    #t1=(t1 + '000')
    #t2=(t2 + '000')
    day1=datetime.strptime(t1, "%d:%m:%Y:%H:%M:%S:%f")
    day2=datetime.strptime(t2, "%d:%m:%Y:%H:%M:%S:%f")
    sec = (day2-day1).total_seconds()
    return(sec)

In [40]:
def extract_keylog_features(path):
    lines = [line.rstrip('\t\n') for line in open(path)]
    f_list = [event.split('\t') for event in lines]
    #lines2 = [line.rstrip('\t\n') for line in open('1.txt')]
    #f_list2 = [event.split('\t') for event in lines2]
    alp = list(string.ascii_lowercase)
    alp2 = list(map(''.join,itertools.combinations(string.ascii_lowercase,2)))

    KeyUps = [x for x in f_list if 'KeyUp' in x]
    KeyDowns = [x for x in f_list if 'KeyDown' in x]

    tups =  [item[2] for item in KeyUps]# if item[1] in alp]
    tdowns =  [item1[2] for item1 in KeyDowns]# if item1[1] in alp]
    try:
        letterup =  [item[1].upper() for item in KeyUps]# if item[1] in alp]
    except: 
        pass

    try:
        letterdown = [item1[1].upper() for item1 in KeyDowns]# if item1[1] in alp]
    except:
        pass

    features = []
    for i in range(0,len(tups)-1):
        t = i

        t1 = tdowns[i] #timestamp of downpress

        if letterup[t] != letterdown[i]: #Simultaneous press
            j = i

            if i == len(tups)-1:
                j = 0
            while j<len(tups)-1 and letterdown[i]!= letterup[j] and i!=len(tups)-1: #skip inbetween to go to the release key
                j = j+1

            tj = tups[j]
            k = i

            if i == 0:
                k = len(tups)-1
            while k>=1 and letterdown[i]!= letterup[k] and i!=0: # repeat same key
                k = k-1

            tk = tups[k]


            if timediff(t1,tk)>0 and timediff(t1,tj)>0 :
                if abs(j-i)<abs(i-k):
                    t = j
                else:
                    t = k

            elif timediff(t1,tk)<0 :
                t = j
            else:
                t = k

        t2 = tups[t]        # release time stamp



        if i!=len(tups)-1:

            t3 = tdowns[i+1]  # Next key time stamp
            latency = timediff(t1,t3)

            lat = letterdown[i]+letterdown[i+1],latency
            features.append(lat)

        hold_time = timediff(t1,t2)
        hold = letterdown[i],hold_time
        features.append(hold)
        
    return features

In [42]:
# Roll_nos = ['17EC32005', '17EC34003', '17EC35023', '17EC35036', '18EC10025', '18EC35010', '18EC35021', '18EC35045', '18EC3AI19']
Roll_nos = ['17EC32005', '17EC34003', '18EC10025', '18EC35010', '18EC35021']
num_roll_nos = len(Roll_nos)

data_features = []
data_tags = []
unique_features = set([])

for r in range(num_roll_nos):
    for subdir,dirs,file in os.walk(os.path.join('Continuous Data Rearranged',Roll_nos[r])):
        for directory in dirs:
#             print(subdir,directory)
            features = extract_keylog_features(os.path.join('Continuous Data Rearranged',Roll_nos[r],directory,'Keylog.txt'))
            data_features.append(features)
            data_tags.append(r)
            
            unique_features = unique_features.union(set([ x for (x,y) in features]))

In [43]:
# unique_features
len(data_features)

70

In [6]:
data_features_temp = []
data_tags_temp = []

splitting_size = 40

for r in range(len(Roll_nos)):
    indices = np.where(np.array(data_tags)==r)[0]
    features_temp = []
    for i in indices:
        features_temp = features_temp + data_features[i]
    
    j=0
    j_next=0
    while(j_next < len(features_temp)):
        j_next = min(j + splitting_size, len(features_temp))
        data_features_temp.append(features_temp[j:j_next])
        data_tags_temp.append(r)
        j = j_next
    
data_features = data_features_temp
data_tags = data_tags_temp

In [7]:
len(data_features_temp)

5243

In [8]:
threshold = 1

for d in range(len(data_features)):
    features_temp = []
    for (x,y) in data_features[d]:
        if(y<threshold):
            features_temp.append((x,y))
    
    data_features[d] = features_temp


In [9]:
unique_features_dict = { list(unique_features)[i]: i for i in range(len(unique_features)) }
num_unique_features = len(unique_features_dict)

count = np.zeros(num_unique_features)
tot_sum = np.zeros(num_unique_features)
tot_square_sum = np.zeros(num_unique_features)
mean = np.zeros(num_unique_features)
avg_square_sum = np.zeros(num_unique_features)
std = np.zeros(num_unique_features)

for sample in data_features:
    for (x,y) in sample:
        idx = unique_features_dict[x]
        count[idx] = count[idx]+1
        tot_sum[idx] = tot_sum[idx] + y
        tot_square_sum[idx] = tot_square_sum[idx] + y*y

for idx in range(len(unique_features_dict)):
    if(count[idx] > 0):
        mean[idx] = tot_sum[idx]/count[idx]
        avg_square_sum[idx] = tot_square_sum[idx]/count[idx]
        std[idx] = np.sqrt(avg_square_sum[idx] - np.square(mean[idx]))

print(mean, avg_square_sum, std)

[0.         0.66330933 0.         ... 0.         0.149908   0.56844133] [0.         0.48787705 0.         ... 0.         0.02247241 0.37550369] [0.         0.21885561 0.         ... 0.         0.         0.22886272]


In [10]:
# Normalization
for d in range(len(data_features)):
    for j in range(len(data_features[d])):
        temp = list(data_features[d][j])
        idx = unique_features_dict[temp[0]]
        if std[idx]>0:
            temp[1] = (temp[1]-mean[idx])/std[idx]
        else:
            temp[1] = 0
        data_features[d][j] = tuple(temp)

In [11]:
shuffling_indices = np.random.permutation(len(data_tags))
shuffled_data = []
shuffled_tags = []

for d in range(len(data_tags)):
    shuffled_data.append(data_features[shuffling_indices[d]])
    shuffled_tags.append(data_tags[shuffling_indices[d]])

In [27]:
class SOM:
    def __init__(self, input_alphabet, num_output_classes):
        self.num_classes = num_output_classes
        self.num_alphabet = len(input_alphabet)
        self.input_alphabet_indices = { list(input_alphabet)[i]: i for i in range(len(input_alphabet)) }
        self.weights = np.random.randn(len(input_alphabet), num_output_classes)
    
    def winning_class(self, sample):
        contender_scores = np.zeros(self.num_classes)
        difference = np.zeros((self.num_alphabet, self.num_classes))
            
        for keystroke, keystroke_val in sample:
            idx = self.input_alphabet_indices[keystroke]
            difference[idx,:] = difference[idx,:] + np.square(self.weights[idx, :] - keystroke_val)
        
        contender_scores = np.sum(difference, 0)
        return np.argmin(contender_scores)
            
    def update_weights(self, sample, winning_class, eta, sigma):
        tot_alphabet_vals = np.zeros(self.num_alphabet)
        alphabet_counts = np.zeros(self.num_alphabet)
        
        for keystroke, keystroke_val in sample:
            idx = self.input_alphabet_indices[keystroke]
            tot_alphabet_vals[idx] = tot_alphabet_vals[idx] + keystroke_val
            alphabet_counts[idx] = alphabet_counts[idx] + 1
        
        Lateral_difference = np.zeros(self.num_classes)
        for keystroke, keystroke_val in sample:
            idx = self.input_alphabet_indices[keystroke]
            Lateral_difference = Lateral_difference + np.square(self.weights[idx,:] - self.weights[idx,winning_class])
        
#         print(Lateral_difference)
        Topo_neigh = np.exp(-Lateral_difference/(2*np.square(sigma)))
        error = 0
        count = 0
        for a in range(self.num_alphabet):
            if(alphabet_counts[a]>0):
                delta_w = (tot_alphabet_vals[a]/alphabet_counts[a] - self.weights[a,:])
                self.weights[a,:] = self.weights[a,:] + eta * Topo_neigh * delta_w
                error = error + np.abs(delta_w[winning_class])
                count = count+1
        error = error/count
        return error
    
    def train(self, training_samples, num_epochs, eta_0, t_eta, sigma_0, t_sigma): 
        
        for t in range(num_epochs):
            eta = eta_0 * np.exp(-t/t_eta)
            sigma = sigma_0 * np.exp(-t/t_sigma)
            error = 0
            for sample in training_samples:
                winning_class = self.winning_class(sample)
                error = error + self.update_weights(sample, winning_class, eta, sigma)
            if(t%2==0):
                print("Finished Epoch: ", t, "\t", error/len(training_samples))
                print(np.linalg.norm(self.weights))

    def supervised_train(self, training_samples, training_tags, learning_rate):        
        for idx, sample in enumerate(training_samples):
            self.update_weights(sample, training_tags[idx], learning_rate)
            
    def calculate_accuracy(self, test_samples, test_tags):
        correct_count = np.zeros((self.num_classes, self.num_classes))
        for idx, sample in enumerate(test_samples):
            winning_class = self.winning_class(sample)
            correct_count[test_tags[idx], winning_class] = correct_count[test_tags[idx], winning_class] + 1
        return correct_count

In [31]:
# hyperparameters
num_epochs = 10
eta_0 = 0.1
t_eta = 5
sigma_0 = 3
t_sigma = 5

In [32]:
OurSOM = SOM(unique_features, 9)

In [33]:
OurSOM.train(shuffled_data, num_epochs, eta_0, t_eta, sigma_0, t_sigma)

Finished Epoch:  0 	 0.6766471513159253
144.6642812090229
Finished Epoch:  2 	 0.6962810883054511
127.91207452348668
Finished Epoch:  4 	 0.694444213528233
122.72102408327935
Finished Epoch:  6 	 0.6779956079180534
121.21332607506967
Finished Epoch:  8 	 0.6506893245208765
121.11874674045436


In [34]:
accuracy = OurSOM.calculate_accuracy(data_features, data_tags)
accuracy

array([[107., 185., 228., 144., 282., 135., 160., 153., 157.],
       [ 84., 101., 126., 232., 273., 321.,  79.,  85.,  87.],
       [ 33.,  45.,  59., 172.,  70., 107.,  37.,  43.,  44.],
       [ 89.,  42.,  94.,  67.,  60.,  47.,  26.,  24.,  42.],
       [108., 118., 123., 177., 200., 244.,  80.,  92.,  61.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [53]:
tot_alphabet_vals = np.zeros(OurSOM.num_alphabet)
alphabet_counts = np.zeros(OurSOM.num_alphabet)

for i in range(50):
    for keystroke, keystroke_val in data_features[i]:
        idx = OurSOM.input_alphabet_indices[keystroke]
        tot_alphabet_vals[idx] = tot_alphabet_vals[idx] + keystroke_val
        alphabet_counts[idx] = alphabet_counts[idx] + 1

errors = 0
count = 0
for a in range(OurSOM.num_alphabet):
    if(alphabet_counts[a]>0):
        tot_alphabet_vals[a] = tot_alphabet_vals[a] / alphabet_counts[a]
        count = count+1
    
print(np.linalg.norm(tot_alphabet_vals)/count)
print(np.std(OurSOM.weights[:,0]))

0.015599744206021138
0.8263432432967903


In [51]:
np.sum(tot_alphabet_vals)/count

0.2177660158206768

In [35]:
for i in range(100):
    AnotherSOM.supervised_train(shuffled_data, shuffled_tags, 0.1)

In [36]:
accuracy = AnotherSOM.calculate_accuracy(data_features, data_tags)
accuracy

array([[529.,  57.,  57.,  41.,  92.],
       [479., 116.,  46.,  16.,  37.],
       [129.,  29., 102.,  18.,  27.],
       [195.,   4.,   3.,  39.,   5.],
       [251.,  67.,  59.,  41., 184.]])

In [36]:
np.linalg.norm(OurSOM.weights[:,1] - OurSOM.weights[:,5])

42.459025537714986

In [109]:
A = { list(unique_features)[i]: i for i in range(len(unique_features))}

In [120]:
np.random.randn()

In [84]:
data_features[0][0:10]

[("'T''E'", 0.077793),
 ("'T'", 0.14801),
 ("'E'KEY.CMD", 6.935901),
 ("'E'", 0.125298),
 ('KEY.CMDKEY.SHIFT', 70.990193),
 ('KEY.CMD', 0.124441),
 ("KEY.SHIFT'M'", 0.312359),
 ('KEY.SHIFT', 0.640483),
 ("'M''T'", 0.062623),
 ("'M'", 0.077998)]

In [29]:
OurSOM.weights

array([[-0.0039801 , -0.52689558, -0.08664363, ..., -0.40036049,
        -0.25177081,  0.60899772],
       [-0.24956706,  0.24995028,  0.07788467, ...,  1.09157211,
         0.8126263 , -0.39892889],
       [ 0.42135002,  1.66713712, -1.50381916, ...,  0.23716661,
         0.7704591 , -0.73510075],
       ...,
       [ 0.32334142, -0.04741357,  0.1700439 , ..., -0.04465147,
         0.81241699,  0.34148805],
       [ 0.72390009,  0.04321274,  0.22706002, ..., -0.22573553,
         0.27111991,  0.54836256],
       [-0.15677389, -0.10171107,  0.27366111, ..., -0.13077741,
         0.90339916, -0.58661632]])

In [35]:
a = np.array(((1,2,3), (4,5,6), (7,8,9)))
b = np.array((1.,2.,3.))
c = np.array((1.,5.,3.))

In [37]:
np.std(a, 0)

array([2.44948974, 2.44948974, 2.44948974])

In [125]:
np.sum(a,0)

array([12, 15, 18])

In [127]:
np.abs([-2,3,4])

array([2, 3, 4])

In [131]:
a[:,0] = [1,1,3]

In [153]:
a = [1,2,3,4,2]
a[0:2]

[1, 2]

In [154]:
np.sum(a,1)

array([ 6, 12, 20])

In [152]:
D = np.array(data_tags)
D[np.where(D==0)]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [76]:
data_features[0]

[("'T''E'", -1.0634524385713477),
 ("'T'", 0.08059210540160572),
 ("'E'", -0.13381387357773233),
 ('KEY.CMD', -0.7446511617692675),
 ("KEY.SHIFT'M'", 1.0928045487958258),
 ('KEY.SHIFT', 1.0017459532551891),
 ("'M''T'", -1.4067165939583643),
 ("'M'", -0.7936816819902973),
 ("'T''P'", -0.7327019522333319),
 ("'T'", 0.14262320208692972),
 ("'P'KEY.SPACE", -0.13785837627661696),
 ("'P'", -0.25348445725879537),
 ("KEY.SPACE'1'", -0.759302969568327),
 ('KEY.SPACE', -0.14038884207309876),
 ("'1'", 0.2511475748313153),
 ("KEY.SHIFT'M'", 2.0366966714836874),
 ('KEY.SHIFT', 0.34944760776364664),
 ("'M''O'", 0.19115912395740817),
 ("'M'", -0.6422334968378073),
 ("'O''D'", -0.9865460860375139),
 ("'O'", -0.2646868938968835),
 ("'D''E'", -0.7575176481208025),
 ("'D'", -0.4393187634793244),
 ("'E''L'", -0.9275041415177712),
 ("'E'", -0.18773305748449112),
 ("'L''L'", -0.6722120517705867),
 ("'L'", -0.7653718178581742),
 ("'L''I'", -0.30508121219579154),
 ("'L'", -0.3647877613728965),
 ("'I''N'", -0.